In [1]:
import pprint
import os
import sys
import numpy as np
from params import get_cfg
from rnn_base import RNN
from mem_sel_rnn import MemRNN
from rnn_base import RNN
from history_rnn import HistoryRNN
from op_selector import OpSel
from mem_selector import MemSel
from ops import Operations
from session import *
from data_gen import samples_generator, split_train_test
import pickle
from data_gen import samples_generator, split_train_test

modelName = 'RNN'
func = 'np_avg_val-5ops'
name = 'state_size_400-num_samples_3500-batch_size_100-learning_rate_0.001-epsilon_0.001-num_features_10-norm_True-softmax_sat_1-clip_False-share_state_True-rnns_same_state_False-state_fn_relu-pen_sofmax_True-smax_pen_r_0.2-seed_24975'
path = '/home/user/neural_program_synthesis/models/summaries/'+modelName+'/'+func+'/'+name+'/'
model_path = path+'/model'
#get the global configuration
cfg_path = path+'cfg.p'
cfg = pickle.load(open(cfg_path, 'rb'))
#instantiate containter with the operations avail for the selection
ops = Operations(cfg)
ops.ops = cfg["used_ops_obj"]
ops.num_of_ops = len(ops.ops)
#generate data 
x,y = samples_generator(cfg['train_fn'], (cfg['num_samples'], cfg['num_features']) , cfg['samples_value_rng'], cfg['seed'])
x_train, x_test, y_train, y_test = split_train_test (x, y , cfg['test_ratio'])

if modelName == "RNN":
        #instantiante the mem selection RNN
        mem = MemRNN(cfg, ops)
        # instanitae the model graph with the main OP selection RNN
        model = RNN(cfg, ops, mem)
        res = restore_selection_matrixes2RNNS(model, cfg, x_train, x_test, y_train, y_test, model_path)
elif modelName == "HistoryRNN":
        #instantiante the mem and op selection
        mem_sel = MemSel(cfg, ops)
        op_sel = OpSel(cfg, ops)
        # instanitae the model graph with the main OP selection RNN
        model = HistoryRNN(cfg, ops, mem_sel, op_sel)
        res = restore_selection_matrixes_HistoryRNNS(model, cfg, x_train, x_test, y_train, y_test, model_path)

timestep 0
timestep 1
timestep 2
timestep 3
timestep 4
timestep 0
timestep 1
timestep 2
timestep 3
timestep 4
[<tf.Variable 'RNN_mem/Params/W_mem:0' shape=(410, 400) dtype=float64_ref>, <tf.Variable 'RNN_mem/Params/b_mem:0' shape=(400,) dtype=float64_ref>, <tf.Variable 'RNN_mem/Params/W2_mem:0' shape=(400, 4) dtype=float64_ref>, <tf.Variable 'RNN_mem/Params/b2_mem:0' shape=(4,) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/W:0' shape=(410, 400) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/b:0' shape=(400,) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/W2:0' shape=(400, 4) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/b2:0' shape=(4,) dtype=float64_ref>]
norming the grads
grads are
[<tf.Tensor 'RNN_op/Grads/clip_by_global_norm/RNN_op/Grads/clip_by_global_norm/_0:0' shape=(410, 400) dtype=float64>, <tf.Tensor 'RNN_op/Grads/clip_by_global_norm/RNN_op/Grads/clip_by_global_norm/_1:0' shape=(400,) dtype=float64>, <tf.Tensor 'RNN_op/Grads/clip_by_global_norm/RNN_op/Grads/clip_by_glob

In [2]:
def print_errors(error_lst, name):
    print("")
    for i, error in enumerate(error_lst):
        print(name + "[" + str(i) + "] err is " + str(error))

In [3]:
def print_matrix(total_error, matrix_lst):
    np.set_printoptions(precision=3, suppress=True)
    print("Error for this matrix is", total_error)
    for elem in range(len(matrix_lst[0])):
        print("\n##Elem-"+str(elem)+"--#############################################################")        
        for matrix in matrix_lst:
            print(matrix[elem], end=" ")
       

In [4]:
def print_ops_matrix(total_error, matrix_lst, ops_list):
    np.set_printoptions(precision=3, suppress=True)
    print("Error for this matrix is", total_error)
    for elem in range(len(matrix_lst[0])):
        print("\n##Elem-"+str(elem)+"--#############################################################")        
        for matrix in matrix_lst:
            index = np.argmax(matrix[elem])
            op_name = ops_list[index].__name__
            print("[ "+op_name+" ]", end=" ")
            

In [5]:
print_errors(res["total_loss_traind_train"], "sofmax_train_error")
print_errors(res["total_loss_traind_test"], "hardmax_train_error")
print_errors(res["total_loss_testd_train"], "softmax_test_error")
print_errors(res["total_loss_testd_test"], "hardmax_test_error")


sofmax_train_error[0] err is 25666.3122458
sofmax_train_error[1] err is 50252.9510372
sofmax_train_error[2] err is 23825.7976059
sofmax_train_error[3] err is 43483.4188541
sofmax_train_error[4] err is 23715.5384432
sofmax_train_error[5] err is 46094.5660853
sofmax_train_error[6] err is 21472.9489641
sofmax_train_error[7] err is 45622.6241733
sofmax_train_error[8] err is 25717.0740528
sofmax_train_error[9] err is 49647.2222023
sofmax_train_error[10] err is 24398.2191698
sofmax_train_error[11] err is 48462.4622969
sofmax_train_error[12] err is 22124.330077
sofmax_train_error[13] err is 46173.242681
sofmax_train_error[14] err is 20973.7885306
sofmax_train_error[15] err is 45860.007876
sofmax_train_error[16] err is 21242.9576925
sofmax_train_error[17] err is 50094.023621
sofmax_train_error[18] err is 20734.9867285
sofmax_train_error[19] err is 48076.2977064
sofmax_train_error[20] err is 23777.8011196
sofmax_train_error[21] err is 48162.5342505
sofmax_train_error[22] err is 18290.2761358



In [6]:
print_matrix(res["total_loss_traind_train"][1], res["softmaxes_traind_train"][1])
#print_ops_matrix(res["total_loss_traind_train"][1], res["softmaxes_traind_train"][1], cfg["used_ops_obj"])

Error for this matrix is 50252.9510372

##Elem-0--#############################################################
[ 1.  0.  0.  0.] [ 1.  0.  0.  0.] [ 1.  0.  0.  0.] [ 1.  0.  0.  0.] [ 1.  0.  0.  0.] 
##Elem-1--#############################################################
[ 1.  0.  0.  0.] [ 1.  0.  0.  0.] [ 1.  0.  0.  0.] [ 1.  0.  0.  0.] [ 1.  0.  0.  0.] 
##Elem-2--#############################################################
[ 1.  0.  0.  0.] [ 1.  0.  0.  0.] [ 1.  0.  0.  0.] [ 1.  0.  0.  0.] [ 1.  0.  0.  0.] 
##Elem-3--#############################################################
[ 1.  0.  0.  0.] [ 1.  0.  0.  0.] [ 1.  0.  0.  0.] [ 1.  0.  0.  0.] [ 1.  0.  0.  0.] 
##Elem-4--#############################################################
[ 1.  0.  0.  0.] [ 1.  0.  0.  0.] [ 1.  0.  0.  0.] [ 1.  0.  0.  0.] [ 1.  0.  0.  0.] 
##Elem-5--#############################################################
[ 1.  0.  0.  0.] [ 1.  0.  0.  0.] [ 1.  0.  0.  0.] [ 1.  0.  0.  0.] [

In [ ]:
print_matrix(res["total_loss_traind_test"][1], res["softmaxes_traind_test"][1])
#print_ops_matrix(res["total_loss_traind_train"][1], res["softmaxes_traind_train_mem"][1], cfg["used_ops_obj"])

In [ ]:
#print_matrix(res["total_loss_traind_train"][1], res["softmaxes_traind_train_mem"][1])
print_ops_matrix(res["total_loss_traind_test"][1], res["softmaxes_traind_train_mem"][1], cfg["used_ops_obj"])

In [ ]:
print_matrix(res["total_loss_traind_train"][1], res["outputs_traind_train_mem"][1])

In [ ]:
res["batchesX_train"][1]

In [ ]:
seed = 10000
x,y = samples_generator(cfg['train_fn'], (cfg['num_samples'], cfg['num_features']) , cfg['samples_value_rng'], seed)
x_train, x_test, y_train, y_test = split_train_test (x, y , cfg['test_ratio'])
res = predict_form_sess(model, cfg, x_test[0:100,], last_hardmax_state_train, model_path, mode="hard")

In [ ]:
res - y_test[0:100,]

In [ ]:
y_test[0:100,]

In [ ]:
res

In [ ]:
len(res["softmaxes_traind_train"])